# Summarizing Text with Amazon Reviews

The objective of this project is to build a model that can create relevant summaries for reviews written about papers.

To build our model we will use a two-layered bidirectional RNN with LSTMs on the input data and two layers, 
each with an LSTM using bahdanau attention on the target data.

The sections of this project are:
- [1.Inspecting the Data](#1.-Insepcting-the-Data)
- [2.Preparing the Data](#2.-Preparing-the-Data)
- [3.Building the Model](#3.-Building-the-Model)
- [4.Training the Model](#4.-Training-the-Model)
- [5.Making Our Own Summaries](#5.-Making-Our-Own-Summaries)

## Download data

word embeddings [numberbatch-en-17.06.txt.gz](https://conceptnet.s3.amazonaws.com/downloads/2017/numberbatch/numberbatch-en-17.06.txt.gz)
after download, extract to **./model/numberbatch-en-17.06.txt**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import tensor_array_ops
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.10.0


In [2]:
import pickle
def __pickleStuff(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def __loadStuff(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

## Load those prepared data and skip to section "[3. Building the Model](#3.-Building-the-Model)"
Once we have run through the "[2.Preparing the Data](#2.-Preparing-the-Data)" section, we should have those data, uncomment and run those lines.

In [3]:
clean_summaries = __loadStuff("./data/clean_summaries.p")
clean_texts = __loadStuff("./data/clean_texts.p")

sorted_summaries = __loadStuff("./data/sorted_summaries.p")
sorted_texts = __loadStuff("./data/sorted_texts.p")
word_embedding_matrix = __loadStuff("./data/word_embedding_matrix.p")

vocab_to_int = __loadStuff("./data/vocab_to_int.p")
int_to_vocab = __loadStuff("./data/int_to_vocab.p")


## 1. Insepcting the Data

In [9]:
reviews = pd.read_csv("Papers.csv")

In [10]:
reviews.shape

(403, 6)

In [11]:
reviews.head()

,Id,Title,EventType,PdfName,Abstract,PaperText
0,5677,Double or Nothing: Multiplicative Incentive Me...,Poster,5677-double-or-nothing-multiplicative-incentiv...,Crowdsourcing has gained immense popularity in...,Double or Nothing: Multiplicative\nIncentive M...
1,5941,Learning with Symmetric Label Noise: The Impor...,Spotlight,5941-learning-with-symmetric-label-noise-the-i...,Convex potential minimisation is the de facto ...,Learning with Symmetric Label Noise: The\nImpo...
2,6019,Algorithmic Stability and Uniform Generalization,Poster,6019-algorithmic-stability-and-uniform-general...,One of the central questions in statistical le...,Algorithmic Stability and Uniform Generalizati...
3,6035,Adaptive Low-Complexity Sequential Inference f...,Poster,6035-adaptive-low-complexity-sequential-infere...,We develop a sequential low-complexity inferen...,Adaptive Low-Complexity Sequential Inference f...
4,5978,Covariance-Controlled Adaptive Langevin Thermo...,Poster,5978-covariance-controlled-adaptive-langevin-t...,Monte Carlo sampling for Bayesian posterior in...,Covariance-Controlled Adaptive Langevin\nTherm...


In [13]:
# Check for any nulls values
reviews.isnull().sum()

Id           0
Title        0
EventType    0
PdfName      0
Abstract     0
PaperText    0
dtype: int64

In [14]:
# Remove null values and unneeded features
reviews = reviews.dropna()
reviews = reviews.drop(['Id','Title','EventType','PdfName'], 1)
reviews = reviews.reset_index(drop=True)

In [15]:
reviews.shape

(403, 2)

In [16]:
reviews.head()

,Abstract,PaperText
0,Crowdsourcing has gained immense popularity in...,Double or Nothing: Multiplicative\nIncentive M...
1,Convex potential minimisation is the de facto ...,Learning with Symmetric Label Noise: The\nImpo...
2,One of the central questions in statistical le...,Algorithmic Stability and Uniform Generalizati...
3,We develop a sequential low-complexity inferen...,Adaptive Low-Complexity Sequential Inference f...
4,Monte Carlo sampling for Bayesian posterior in...,Covariance-Controlled Adaptive Langevin\nTherm...


In [30]:
# Inspecting some of the reviews
for i in range(5):
    print("Paper #",i+1)
    print(reviews.Abstract[i]+'\n')
    print(len(reviews.PaperText[i]))
    #print()

Paper # 1
Crowdsourcing has gained immense popularity in machine learning applications for obtaining large amounts of labeled data. Crowdsourcing is cheap and fast, but suffers from the problem of low-quality data. To address this fundamental challenge in crowdsourcing, we propose a simple payment mechanism to incentivize workers to answer only the questions that they are sure of and skip the rest. We show that surprisingly, under a mild and natural no-free-lunch requirement, this mechanism is the one and only incentive-compatible payment mechanism possible. We also show that among all possible incentive-compatible  mechanisms (that may or may not satisfy no-free-lunch), our mechanism makes the smallest possible payment to spammers.  Interestingly, this unique mechanism takes a multiplicative form. The simplicity of the mechanism is an added benefit.  In preliminary experiments involving over several hundred workers, we observe a significant reduction in the error rates under our uniqu

## 2. Preparing the Data

In [31]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [32]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        # We are not using "text.split()" here
        #since it is not fool proof, e.g. words followed by punctuations "Are you kidding?I think you aren't."
        text = re.findall(r"[\w']+", text)
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)# remove links
    text = re.sub(r'\<a href', ' ', text)# remove html link tag
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [33]:
clean_text("That's a great movie,Can you believe it?I've.But you may not.")

'great movie believe may'

### Clean the summaries and texts
We will remove the stopwords from the texts because they do not provide much use for training our model. However, we will keep them for our summaries so that they sound more like natural phrases. 

In [35]:
clean_summaries = []
for summary in reviews.Abstract:
    clean_summaries.append(clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in reviews.PaperText:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

Summaries are complete.
Texts are complete.


In [37]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i]+'\n')
    print(clean_texts[i])
    print()

Clean Review # 1
crowdsourcing has gained immense popularity in machine learning applications for obtaining large amounts of labeled data crowdsourcing is cheap and fast but suffers from the problem of low quality data to address this fundamental challenge in crowdsourcing we propose a simple payment mechanism to incentivize workers to answer only the questions that they are sure of and skip the rest we show that surprisingly under a mild and natural no free lunch requirement this mechanism is the one and only incentive compatible payment mechanism possible we also show that among all possible incentive compatible mechanisms that may or may not satisfy no free lunch our mechanism makes the smallest possible payment to spammers interestingly this unique mechanism takes a multiplicative form the simplicity of the mechanism is an added benefit in preliminary experiments involving over several hundred workers we observe a significant reduction in the error rates under our unique mechanism 

### Count the number of occurrences of each word in a set of text

In [38]:
def count_words(count_dict, text):
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

#### Give the function a try

In [39]:
mydict = {}
count_words(mydict, ["that is a great great great dog","you have a great dog"])
mydict

{'that': 1, 'is': 1, 'a': 2, 'great': 4, 'dog': 2, 'you': 1, 'have': 1}

In [40]:
word_counts = {}
count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 47962


Let's see how may "hero" occurs in the data

In [42]:
word_counts["computer"]

603

### Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
 (https://github.com/commonsense/conceptnet-numberbatch)

In [44]:

embeddings_index = {}
with open('/Users/apple/Desktop/601Project-master/numberbatch-en-17.06.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 417195


### Take a look at the CN embedding dimension

In [53]:
embeddings_index["test"].shape

(300,)

### Find the number of words that are missing from CN, and are used more than our threshold.

I use a **threshold** of 20, so that words not in CN can be added to our **word_embedding_matrix**, but they need to be common enough in the reviews so that the model can understand their meaning.

In [57]:
missing_words = 0
threshold = 20

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 1266
Percent of words that are missing from vocabulary: 2.64%


### What are those missing words in the CN
Looks mostly products' brand.

In [59]:
missing_words = []
for word, count in word_counts.items():
    if count > threshold and word not in embeddings_index:
        missing_words.append((word,count))
missing_words[:30]

[('servedio', 25),
 ('2008', 449),
 ('sln', 91),
 ('svm', 287),
 ('dirichlet', 245),
 ('gaussians', 117),
 ('hyperparameters', 149),
 ('subsampling', 76),
 ('langevin', 69),
 ('quantifies', 40),
 ('multiclass', 127),
 ('leverages', 30),
 ('convolutional', 748),
 ('crcn', 27),
 ('outperforms', 309),
 ('kwikcluster', 25),
 ('clusterwild', 49),
 ('speedups', 35),
 ('32', 575),
 ('objectness', 25),
 ('vgg', 90),
 ('16', 1615),
 ('2007', 449),
 ('73', 127),
 ('2012', 1035),
 ('70', 230),
 ('300', 258),
 ('github', 67),
 ('rcnn', 78),
 ('datasets', 1091)]

### Words to indexes, indexes to words dicts
Limit the vocab that we will use to words that appear ≥ threshold or are in CN

In [63]:
#dictionary to convert words to integers
vocab_to_int = {} 
# Index words from 0
value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 47962
Number of words we will use: 21556
Percent of words we will use: 44.940000000000005%


### Create word embedding matrix
It has shape (nb_words, embedding_dim) i.e. (59072, 300) in this case. 1st dim is word index, 2nd dim is from CN or random generated.

In [67]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)


# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

21556


### Function to convert sentences to sequence of words indexes
It also use `<UNK>` index to replace unknown words, append `<EOS>` (End of Sentence) to the sequences if eos is set True

In [68]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

Apply convert_to_ints to clean_summaries and clean_texts

In [69]:

word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count,  eos=True)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 1623654
Total number of UNKs in headlines: 73211
Percent of words that are UNK: 4.51%


### Take a look at what the sequence looks like
Each number here represents a word

In [75]:
len(int_summaries[20])

117

### Function to get the length of each sequence

In [76]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [77]:
create_lengths(int_summaries[:3])

,counts
0,159
1,152
2,168


Get statistic summary of the length of summaries and texts

In [78]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
           counts
count  403.000000
mean   154.759305
std     45.652038
min     62.000000
25%    123.000000
50%    148.000000
75%    184.000000
max    349.000000

Texts:
            counts
count   403.000000
mean   3876.158809
std     392.740745
min    2846.000000
25%    3604.500000
50%    3869.000000
75%    4161.000000
max    6094.000000


### See what's the max squence length we can cover by percentile

In [80]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 89.5))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

4360.58
4457.7
4761.8


In [81]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

210.60000000000002
240.69999999999993
273.9200000000001


## Function to counts the number of time `<UNK>` appears in a sentence

In [82]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

**Filter** for length limit and number of `<UNK>`s

**Sort** the summaries and texts by the length of the element in **texts** from shortest to longest


In [124]:
max_text_length = 4361 # This will cover up to 89.5% lengthes
max_summary_length = 274 # This will cover up to 99% lengthes
min_length = 2
unk_text_limit = 240 # text can contain up to 1 UNK word
unk_summary_limit = 12 # Summary should not contain any UNK word

def filter_condition(item):
    int_summary = item[0]
    int_text = item[1]
    if(len(int_summary) >= min_length and 
       len(int_summary) <= max_summary_length and 
       len(int_text) >= min_length and 
       len(int_text) <= max_text_length and 
       unk_counter(int_summary) <= unk_summary_limit and 
       unk_counter(int_text) <= unk_text_limit):
        return True
    else:
        return False

int_text_summaries = list(zip(int_summaries , int_texts))
int_text_summaries_filtered = list(filter(filter_condition, int_text_summaries))
print(len(int_text_summaries_filtered))
sorted_int_text_summaries = sorted(int_text_summaries_filtered, key=lambda item: len(item[1]))
sorted_int_text_summaries = list(zip(*sorted_int_text_summaries))
sorted_summaries = list(sorted_int_text_summaries[0])
sorted_texts = list(sorted_int_text_summaries[1])
#Delete those temporary varaibles
del int_text_summaries, sorted_int_text_summaries, int_text_summaries_filtered
#Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

314
314
314


### Inspect the length of text in sorted_texts

In [125]:
lengths_texts = [len(text) for text in sorted_texts]
lengths_texts[:20]

[2846,
 2956,
 2979,
 3008,
 3033,
 3034,
 3042,
 3077,
 3128,
 3138,
 3186,
 3190,
 3190,
 3206,
 3221,
 3230,
 3236,
 3237,
 3264,
 3267]

## Save data for later

In [127]:
__pickleStuff("/Users/apple/Desktop/601Project-master/clean_summaries.p",clean_summaries)
__pickleStuff("/Users/apple/Desktop/601Project-master/clean_texts.p",clean_texts)

__pickleStuff("/Users/apple/Desktop/601Project-master/sorted_summaries.p",sorted_summaries)
__pickleStuff("/Users/apple/Desktop/601Project-master/sorted_texts.p",sorted_texts)
__pickleStuff("/Users/apple/Desktop/601Project-master/word_embedding_matrix.p",word_embedding_matrix)

__pickleStuff("/Users/apple/Desktop/601Project-master/vocab_to_int.p",vocab_to_int)
__pickleStuff("/Users/apple/Desktop/601Project-master/int_to_vocab.p",int_to_vocab)


## 3. Building the Model

Create palceholders for inputs to the model

**summary_length** and **text_length** are the sentence lengths in a batch, and **max_summary_length** is the maximum length of a summary in a batch.

In [133]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

Remove the last word id from each batch and concatenate the id of `<GO>` to the begining of each batch

In [134]:
def process_encoding_input(target_data, vocab_to_int, batch_size):  
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1]) # slice it to target_data[0:batch_size, 0: -1]
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

### Create the encoding layers

bidirectional_dynamic_rnn
use **tf.variable_scope** so that variables are reused with each layer

parameters
- **rnn_size**: The number of units in the LSTM cell
- **sequence_length**: size [batch_size], containing the actual lengths for each of the sequences in the batch
- **num_layers**: number of bidirectional RNN layer
- **rnn_inputs**: number of bidirectional RNN layer
- **keep_prob**: RNN dropout input keep probability

In [141]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
            enc_output = tf.concat(enc_output,2)
            # original code is missing this line below, that is how we connect layers 
            # by feeding the current layer's output to next layer's input
            rnn_inputs = enc_output
    return enc_output, enc_state

### Create the training decoding layer
parameters
- **dec_embed_input**: output of embedding_lookup for a batch of inputs
- **summary_length**: length of each padded summary sequences in batch, since padded, all lengths should be same number 
- **dec_cell**: the decoder RNN cells' output with attention wapper
- **output_layer**: fully connected layer to apply to the RNN output
- **vocab_size**: vocabulary size i.e. len(vocab_to_int)+1
- **max_summary_length**: the maximum length of a summary in a batch
- **batch_size**: number of input sequences in a batch

Three components

- **TraingHelper** reads a sequence of integers from the encoding layer.
- **BasicDecoder** processes the sequence with the decoding cell, and an output layer, which is a fully connected layer. **initial_state** set to zero state.
- **dynamic_decode** creates our outputs that will be used for training.

In [142]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, output_layer,
                            vocab_size, max_summary_length,batch_size):
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                                       helper=training_helper,
                                                       initial_state=dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                       output_layer = output_layer)

    training_logits = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

### Create infer decoding layer

parameters
- **embeddings**: the CN's word_embedding_matrix
- **start_token**: the id of `<GO>`
- **end_token**: the id of `<EOS>`
- **dec_cell**: the decoder RNN cells' output with attention wapper
- **output_layer**: fully connected layer to apply to the RNN output
- **max_summary_length**: the maximum length of a summary in a batch
- **batch_size**: number of input sequences in a batch

**GreedyEmbeddingHelper** argument **start_tokens**: int32 vector shaped [batch_size], the start tokens.

In [146]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size),
                                                        output_layer)
                
    inference_logits = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

### Create Decoding layer
3 parts: decoding cell, attention, and getting our logits.
#### Decoding Cell: 
Just a two layer LSTM with dropout.
#### Attention: 
Using Bhadanau, since trains faster than Luong. 

**AttentionWrapper** applies the attention mechanism to our decoding cell.

parameters
- **dec_embed_input**: output of embedding_lookup for a batch of inputs
- **embeddings**: the CN's word_embedding_matrix
- **enc_output**: encoder layer output, containing the forward and the backward rnn output
- **enc_state**: encoder layer state, a tuple containing the forward and the backward final states of bidirectional rnn.
- **vocab_size**: vocabulary size i.e. len(vocab_to_int)+1
- **text_length**: the actual lengths for each of the input text sequences in the batch
- **summary_length**: the actual lengths for each of the input summary sequences in the batch
- **max_summary_length**: the maximum length of a summary in a batch
- **rnn_size**: The number of units in the LSTM cell
- **vocab_to_int**: vocab_to_int the dictionary
- **keep_prob**: RNN dropout input keep probability
- **batch_size**: number of input sequences in a batch
- **num_layers**: number of decoder RNN layer

In [147]:
def lstm_cell(lstm_size, keep_prob):
    cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    return tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob = keep_prob)

def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length,
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    dec_cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    output_layer = Dense(vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1))
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                     enc_output,
                                                     text_length,
                                                     normalize=False,
                                                     name='BahdanauAttention')
    dec_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell,attn_mech,rnn_size)
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input,summary_length,dec_cell,
                                                  output_layer,
                                                  vocab_size,
                                                  max_summary_length,
                                                  batch_size)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,
                                                    vocab_to_int['<GO>'],
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell,
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)
    return training_logits, inference_logits

In [148]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size) #shape=(batch_size, senquence length) each seq start with index of<GO>
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    return training_logits, inference_logits

### Pad sentences for batch
Pad so the actual lengths for each of the sequences in the batch have the same length.

In [149]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

### Function to generate batch data for training

In [153]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

#### Just to test "get_batches" function
Here we generate a batch with size of 5

Checkout those "59069" they are `<PAD>`s, also all sequences' lengths are the same.

In [154]:
print("'<PAD>' has id: {}".format(vocab_to_int['<PAD>']))
sorted_summaries_samples = sorted_summaries[7:50]
sorted_texts_samples = sorted_texts[7:50]
pad_summaries_batch_samples, pad_texts_batch_samples, pad_summaries_lengths_samples, pad_texts_lengths_samples = next(get_batches(
    sorted_summaries_samples, sorted_texts_samples, 5))
print("pad summaries batch samples:\n\r {}".format(pad_summaries_batch_samples))

'<PAD>' has id: 21553
pad summaries batch samples:
 [[  723   275   724   725    27   726   727   350     5   412   728    18
     27   729    23   727   730    13   731    22   732   495    32    33
     34   733   511     9   734   301   735    13   727   736   737    22
    279   738   739    18   495   145    23   740    13    70   733   511
     16    34   741   652   742   743   277    80   592   346   664    13
    727   730   629   744   619   741   745    18   746    32   119   729
    301   747   730   444   748   749   750    13   751   170   752   753
      5    23   652   742    27   754   755   610    13   473   183    32
    269   454   512   756   473   757    27   727   730   758    34   759
     13    23   733   187   444   624     7    32   760    70   511   761
     18   762   133   277    34   763   764   207    35   765   766   767
    768   581    49    43   761    16   769    27   729    23   727   730
     13   731    18   726    23   770    13    34   771    1

In [156]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.95

## Build graph

In [157]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits[0].rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits[0].sample_id, name='predictions')
    
    # Create the weights for sequence_loss, the sould be all True across since each batch is padded
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
Graph is built.
./graph


## 4. Training the Model

Only going to use a subset of the data to reduce the traing time for this demo.

We chose not use use the start of the subset because because those are shorter sequences and we don't want to make it too easy for the model.

In [160]:
# Subset the data for training
start = 200
end = start + 50
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 3935
The longest text length: 4094


In [58]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005

display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                #else:
                    #print("No Improvement.")
                    #stop_early += 1
                    #if stop_early == stop:
                        #break
                #update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        #if stop_early == stop:
            #print("Stopping Training.")
            #break

Epoch   1/100 Batch   20/781 - Loss:  4.931, Seconds: 2.67
Epoch   1/100 Batch   40/781 - Loss:  2.845, Seconds: 2.23
Epoch   1/100 Batch   60/781 - Loss:  2.871, Seconds: 2.77
Epoch   1/100 Batch   80/781 - Loss:  2.816, Seconds: 2.67
Epoch   1/100 Batch  100/781 - Loss:  2.682, Seconds: 2.66
Epoch   1/100 Batch  120/781 - Loss:  2.695, Seconds: 2.49
Epoch   1/100 Batch  140/781 - Loss:  2.583, Seconds: 2.59
Epoch   1/100 Batch  160/781 - Loss:  2.839, Seconds: 2.20
Epoch   1/100 Batch  180/781 - Loss:  2.695, Seconds: 2.52
Epoch   1/100 Batch  200/781 - Loss:  2.667, Seconds: 2.84
Epoch   1/100 Batch  220/781 - Loss:  2.603, Seconds: 2.63
Epoch   1/100 Batch  240/781 - Loss:  2.456, Seconds: 2.62
Average loss for this update: 2.866
New Record!
Epoch   1/100 Batch  260/781 - Loss:  2.558, Seconds: 2.74
Epoch   1/100 Batch  280/781 - Loss:  2.639, Seconds: 2.73
Epoch   1/100 Batch  300/781 - Loss:  2.651, Seconds: 2.87
Epoch   1/100 Batch  320/781 - Loss:  2.672, Seconds: 2.67
Epoch   

Epoch   4/100 Batch  300/781 - Loss:  1.737, Seconds: 2.70
Epoch   4/100 Batch  320/781 - Loss:  1.818, Seconds: 2.71
Epoch   4/100 Batch  340/781 - Loss:  1.626, Seconds: 2.66
Epoch   4/100 Batch  360/781 - Loss:  1.688, Seconds: 2.80
Epoch   4/100 Batch  380/781 - Loss:  1.477, Seconds: 2.91
Epoch   4/100 Batch  400/781 - Loss:  1.614, Seconds: 2.88
Epoch   4/100 Batch  420/781 - Loss:  1.591, Seconds: 2.59
Epoch   4/100 Batch  440/781 - Loss:  1.705, Seconds: 2.94
Epoch   4/100 Batch  460/781 - Loss:  1.781, Seconds: 2.85
Epoch   4/100 Batch  480/781 - Loss:  1.582, Seconds: 2.66
Epoch   4/100 Batch  500/781 - Loss:  1.619, Seconds: 2.70
Average loss for this update: 1.64
New Record!
Epoch   4/100 Batch  520/781 - Loss:  1.492, Seconds: 2.54
Epoch   4/100 Batch  540/781 - Loss:  1.559, Seconds: 2.80
Epoch   4/100 Batch  560/781 - Loss:  1.491, Seconds: 2.61
Epoch   4/100 Batch  580/781 - Loss:  1.728, Seconds: 2.79
Epoch   4/100 Batch  600/781 - Loss:  1.766, Seconds: 2.76
Epoch   4

Epoch   7/100 Batch  580/781 - Loss:  1.312, Seconds: 2.74
Epoch   7/100 Batch  600/781 - Loss:  1.317, Seconds: 2.76
Epoch   7/100 Batch  620/781 - Loss:  1.245, Seconds: 2.68
Epoch   7/100 Batch  640/781 - Loss:  1.183, Seconds: 2.76
Epoch   7/100 Batch  660/781 - Loss:  1.045, Seconds: 2.87
Epoch   7/100 Batch  680/781 - Loss:  1.133, Seconds: 2.78
Epoch   7/100 Batch  700/781 - Loss:  1.268, Seconds: 2.83
Epoch   7/100 Batch  720/781 - Loss:  1.444, Seconds: 3.06
Epoch   7/100 Batch  740/781 - Loss:  1.289, Seconds: 3.03
Epoch   7/100 Batch  760/781 - Loss:  1.277, Seconds: 2.78
Average loss for this update: 1.222
New Record!
Epoch   7/100 Batch  780/781 - Loss:  1.091, Seconds: 2.56
Epoch   8/100 Batch   20/781 - Loss:  1.353, Seconds: 2.67
Epoch   8/100 Batch   40/781 - Loss:  1.241, Seconds: 2.28
Epoch   8/100 Batch   60/781 - Loss:  1.210, Seconds: 2.86
Epoch   8/100 Batch   80/781 - Loss:  1.214, Seconds: 2.67
Epoch   8/100 Batch  100/781 - Loss:  1.058, Seconds: 2.69
Epoch   

Epoch  11/100 Batch   80/781 - Loss:  0.932, Seconds: 2.71
Epoch  11/100 Batch  100/781 - Loss:  0.815, Seconds: 2.81
Epoch  11/100 Batch  120/781 - Loss:  0.893, Seconds: 2.48
Epoch  11/100 Batch  140/781 - Loss:  0.853, Seconds: 2.59
Epoch  11/100 Batch  160/781 - Loss:  0.985, Seconds: 2.20
Epoch  11/100 Batch  180/781 - Loss:  0.957, Seconds: 2.50
Epoch  11/100 Batch  200/781 - Loss:  0.909, Seconds: 2.77
Epoch  11/100 Batch  220/781 - Loss:  0.913, Seconds: 2.63
Epoch  11/100 Batch  240/781 - Loss:  0.765, Seconds: 2.64
Average loss for this update: 0.91
New Record!
Epoch  11/100 Batch  260/781 - Loss:  0.857, Seconds: 2.73
Epoch  11/100 Batch  280/781 - Loss:  0.877, Seconds: 2.78
Epoch  11/100 Batch  300/781 - Loss:  0.974, Seconds: 2.67
Epoch  11/100 Batch  320/781 - Loss:  1.015, Seconds: 3.00
Epoch  11/100 Batch  340/781 - Loss:  0.900, Seconds: 2.71
Epoch  11/100 Batch  360/781 - Loss:  0.896, Seconds: 2.80
Epoch  11/100 Batch  380/781 - Loss:  0.788, Seconds: 2.90
Epoch  11

Epoch  14/100 Batch  360/781 - Loss:  0.725, Seconds: 2.76
Epoch  14/100 Batch  380/781 - Loss:  0.631, Seconds: 2.90
Epoch  14/100 Batch  400/781 - Loss:  0.717, Seconds: 2.89
Epoch  14/100 Batch  420/781 - Loss:  0.661, Seconds: 2.54
Epoch  14/100 Batch  440/781 - Loss:  0.690, Seconds: 2.96
Epoch  14/100 Batch  460/781 - Loss:  0.747, Seconds: 2.84
Epoch  14/100 Batch  480/781 - Loss:  0.705, Seconds: 2.62
Epoch  14/100 Batch  500/781 - Loss:  0.745, Seconds: 2.75
Average loss for this update: 0.714
New Record!
Epoch  14/100 Batch  520/781 - Loss:  0.648, Seconds: 2.47
Epoch  14/100 Batch  540/781 - Loss:  0.701, Seconds: 2.95
Epoch  14/100 Batch  560/781 - Loss:  0.647, Seconds: 2.61
Epoch  14/100 Batch  580/781 - Loss:  0.720, Seconds: 2.76
Epoch  14/100 Batch  600/781 - Loss:  0.733, Seconds: 2.95
Epoch  14/100 Batch  620/781 - Loss:  0.690, Seconds: 2.65
Epoch  14/100 Batch  640/781 - Loss:  0.708, Seconds: 2.83
Epoch  14/100 Batch  660/781 - Loss:  0.583, Seconds: 2.86
Epoch  1

...

Epoch  21/100 Batch  140/781 - Loss:  0.427, Seconds: 2.57
Epoch  21/100 Batch  160/781 - Loss:  0.458, Seconds: 2.16
Epoch  21/100 Batch  180/781 - Loss:  0.463, Seconds: 2.42
Epoch  21/100 Batch  200/781 - Loss:  0.445, Seconds: 2.76
Epoch  21/100 Batch  220/781 - Loss:  0.470, Seconds: 2.60
Epoch  21/100 Batch  240/781 - Loss:  0.375, Seconds: 2.63
Average loss for this update: 0.451
New Record!
Epoch  21/100 Batch  260/781 - Loss:  0.436, Seconds: 2.87
Epoch  21/100 Batch  280/781 - Loss:  0.421, Seconds: 2.75
Epoch  21/100 Batch  300/781 - Loss:  0.486, Seconds: 2.65
Epoch  21/100 Batch  320/781 - Loss:  0.483, Seconds: 2.68
Epoch  21/100 Batch  340/781 - Loss:  0.449, Seconds: 2.69
Epoch  21/100 Batch  360/781 - Loss:  0.436, Seconds: 2.80
Epoch  21/100 Batch  380/781 - Loss:  0.396, Seconds: 2.89
Epoch  21/100 Batch  400/781 - Loss:  0.453, Seconds: 2.89
Epoch  21/100 Batch  420/781 - Loss:  0.408, Seconds: 2.61
Epoch  21/100 Batch  440/781 - Loss:  0.447, Seconds: 3.08
Epoch  2

Epoch  24/100 Batch  420/781 - Loss:  0.357, Seconds: 2.55
Epoch  24/100 Batch  440/781 - Loss:  0.390, Seconds: 2.97
Epoch  24/100 Batch  460/781 - Loss:  0.393, Seconds: 2.86
Epoch  24/100 Batch  480/781 - Loss:  0.383, Seconds: 2.63
Epoch  24/100 Batch  500/781 - Loss:  0.423, Seconds: 2.76
Average loss for this update: 0.382
No Improvement.
Epoch  24/100 Batch  520/781 - Loss:  0.334, Seconds: 2.50
Epoch  24/100 Batch  540/781 - Loss:  0.373, Seconds: 2.84
Epoch  24/100 Batch  560/781 - Loss:  0.362, Seconds: 2.55
Epoch  24/100 Batch  580/781 - Loss:  0.386, Seconds: 2.80
Epoch  24/100 Batch  600/781 - Loss:  0.392, Seconds: 3.01
Epoch  24/100 Batch  620/781 - Loss:  0.388, Seconds: 2.62
Epoch  24/100 Batch  640/781 - Loss:  0.383, Seconds: 2.80
Epoch  24/100 Batch  660/781 - Loss:  0.325, Seconds: 2.88
Epoch  24/100 Batch  680/781 - Loss:  0.375, Seconds: 2.77
Epoch  24/100 Batch  700/781 - Loss:  0.409, Seconds: 2.78
Epoch  24/100 Batch  720/781 - Loss:  0.450, Seconds: 3.10
Epoc

Epoch  27/100 Batch  700/781 - Loss:  0.342, Seconds: 2.82
Epoch  27/100 Batch  720/781 - Loss:  0.374, Seconds: 2.89
Epoch  27/100 Batch  740/781 - Loss:  0.338, Seconds: 3.06
Epoch  27/100 Batch  760/781 - Loss:  0.348, Seconds: 2.82
Average loss for this update: 0.329
New Record!
Epoch  27/100 Batch  780/781 - Loss:  0.313, Seconds: 2.62
Epoch  28/100 Batch   20/781 - Loss:  0.385, Seconds: 2.67
Epoch  28/100 Batch   40/781 - Loss:  0.315, Seconds: 2.25
Epoch  28/100 Batch   60/781 - Loss:  0.332, Seconds: 2.83
Epoch  28/100 Batch   80/781 - Loss:  0.312, Seconds: 2.79
Epoch  28/100 Batch  100/781 - Loss:  0.293, Seconds: 2.72
Epoch  28/100 Batch  120/781 - Loss:  0.315, Seconds: 2.49
Epoch  28/100 Batch  140/781 - Loss:  0.302, Seconds: 2.60
Epoch  28/100 Batch  160/781 - Loss:  0.325, Seconds: 2.18
Epoch  28/100 Batch  180/781 - Loss:  0.318, Seconds: 2.42
Epoch  28/100 Batch  200/781 - Loss:  0.315, Seconds: 2.74
Epoch  28/100 Batch  220/781 - Loss:  0.327, Seconds: 2.66
Epoch  2

Epoch  31/100 Batch  200/781 - Loss:  0.271, Seconds: 2.76
Epoch  31/100 Batch  220/781 - Loss:  0.290, Seconds: 2.69
Epoch  31/100 Batch  240/781 - Loss:  0.230, Seconds: 2.63
Average loss for this update: 0.282
New Record!
Epoch  31/100 Batch  260/781 - Loss:  0.280, Seconds: 2.81
Epoch  31/100 Batch  280/781 - Loss:  0.262, Seconds: 2.74
Epoch  31/100 Batch  300/781 - Loss:  0.290, Seconds: 2.72
Epoch  31/100 Batch  320/781 - Loss:  0.304, Seconds: 2.83
Epoch  31/100 Batch  340/781 - Loss:  0.272, Seconds: 2.71
Epoch  31/100 Batch  360/781 - Loss:  0.283, Seconds: 2.81
Epoch  31/100 Batch  380/781 - Loss:  0.232, Seconds: 2.89
Epoch  31/100 Batch  400/781 - Loss:  0.272, Seconds: 2.87
Epoch  31/100 Batch  420/781 - Loss:  0.243, Seconds: 2.61
Epoch  31/100 Batch  440/781 - Loss:  0.268, Seconds: 2.95
Epoch  31/100 Batch  460/781 - Loss:  0.288, Seconds: 2.90
Epoch  31/100 Batch  480/781 - Loss:  0.283, Seconds: 2.67
Epoch  31/100 Batch  500/781 - Loss:  0.319, Seconds: 2.69
Average 

Epoch  34/100 Batch  480/781 - Loss:  0.258, Seconds: 2.61
Epoch  34/100 Batch  500/781 - Loss:  0.282, Seconds: 2.75
Average loss for this update: 0.25
New Record!
Epoch  34/100 Batch  520/781 - Loss:  0.224, Seconds: 2.48
Epoch  34/100 Batch  540/781 - Loss:  0.245, Seconds: 3.01
Epoch  34/100 Batch  560/781 - Loss:  0.224, Seconds: 2.78
Epoch  34/100 Batch  580/781 - Loss:  0.267, Seconds: 2.76
Epoch  34/100 Batch  600/781 - Loss:  0.259, Seconds: 2.76
Epoch  34/100 Batch  620/781 - Loss:  0.244, Seconds: 2.61
Epoch  34/100 Batch  640/781 - Loss:  0.254, Seconds: 2.76
Epoch  34/100 Batch  660/781 - Loss:  0.218, Seconds: 2.89
Epoch  34/100 Batch  680/781 - Loss:  0.240, Seconds: 2.76
Epoch  34/100 Batch  700/781 - Loss:  0.271, Seconds: 2.81
Epoch  34/100 Batch  720/781 - Loss:  0.284, Seconds: 2.93
Epoch  34/100 Batch  740/781 - Loss:  0.259, Seconds: 3.10
Epoch  34/100 Batch  760/781 - Loss:  0.283, Seconds: 2.80
Average loss for this update: 0.253
No Improvement.
Epoch  34/100 Ba

## 5. Making Our Own Summaries

To see the quality of the summaries that this model can generate, you can either create your own review, or use a review from the dataset. You can set the length of the summary to a fixed value, or use a random value like I have here.

In [59]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]


- **input_sentences**: a list of reviews strings we are going to summarize
- **generagte_summary_length**: a int or list, if a list must be same length as input_sentences


In [65]:
input_sentences=["If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal.",
                 "This case is being advertised on Amazon that it fits an IPhone XS 2018.It does not fit my IPhone XS 2018.This I consider poor customer service on Spigen’s par"]
#input_sentences=["If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal."]
generagte_summary_length =  [3,2]

texts = [text_to_seq(input_sentence) for input_sentence in input_sentences]
checkpoint = "./best_model.ckpt"
if type(generagte_summary_length) is list:
    if len(input_sentences)!=len(generagte_summary_length):
        raise Exception("[Error] makeSummaries parameter generagte_summary_length must be same length as input_sentences or an integer")
    generagte_summary_length_list = generagte_summary_length
else:
    generagte_summary_length_list = [generagte_summary_length] * len(texts)
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    #Multiply by batch_size to match the model's input parameters
    for i, text in enumerate(texts):
        generagte_summary_length = generagte_summary_length_list[i]
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [generagte_summary_length], #summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 
        # Remove the padding from the summaries
        pad = vocab_to_int["<PAD>"] 
        print('- Review:\n\r {}'.format(input_sentences[i]))
        print('- Summary:\n\r {}\n\r\n\r'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
- Review:
 If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal.
- Summary:
 great product


- Review:
 This case is being advertised on Amazon that it fits an IPhone XS 2018.It does not fit my IPhone XS 2018.This I consider poor customer service on Spigen’s par
- Summary:
 a decent




## Summary

I hope that you found this project to be rather interesting and informative. One of my main recommendations for working with this dataset and model is either use a GPU, a subset of the dataset, or plenty of time to train your model. As you might be able to expect, the model will not be able to make good predictions just by seeing many reviews, it needs so see the reviews many times to be able to understand the relationship between words and between descriptions & summaries. 

In short, I'm pleased with how well this model performs. After creating numerous reviews and checking those from the dataset, I can happily say that most of the generated summaries are appropriate, some of them are great, and some of them make mistakes. I'll try to improve this model and if it gets better, I'll update my GitHub.

Thanks for reading!